In [ ]:
import numpy as np
import os

np.random.seed(42)

from terrainbento import Basic

params = {
    # create the Clock.
    "clock": {"start": 0, 
              "step": 100,
              "stop": 1e3}, # the real readme uses 1e5, we use 1e3 for testing
    
    # Create the Grid
    "grid": {
        "RasterModelGrid": [
            (25, 40), # the real readme uses (200, 320), here we use a smaller grid so testing works quickly
            {
                "xy_spacing": 10
            },
            {
                "fields": {
                    "node": {
                        "topographic__elevation": {
                            "random": [{
                                "where": "CORE_NODE"
                            }]
                        }
                    }
                }
            },
        ]
    },
    
    # Set up Boundary Handlers
    "boundary_handlers":{"NotCoreNodeBaselevelHandler": {"modify_core_nodes": True, 
                                                         "lowering_rate": -0.001}},
    # Parameters that control output.
    "output_interval": 1e3, 
    "save_first_timestep": True,
    "output_prefix": "readme_example",
    "fields":["topographic__elevation"],
             
    # Parameters that control process and rates.
    "water_erodibility" : 0.001,
    "m_sp" : 0.5,
    "n_sp" : 1.0, 
    "regolith_transport_parameter" : 0.2,           
         }

model = Basic.from_dict(params)

In [ ]:
model.run()

In [ ]:
from landlab import imshow_grid

filenames = []
with model.to_xarray_dataset() as ds:
    for i in range(ds.topographic__elevation.shape[0]):
        filename = "temp_output."+str(i)+".png"
        imshow_grid(model.grid, 
                    ds.topographic__elevation.values[i, :, :], 
                    cmap="viridis", 
                    limits=(0, 12), 
                    output=filename)
        filenames.append(filename)
del ds

In [ ]:
# to create a gif, install imageio and use the following code.

# import imageio
# with imageio.get_writer("terrainbento_example.gif", mode="I") as writer:
#     for filename in filenames:
#         image = imageio.imread(filename)
#         writer.append_data(image)
#         os.remove(filename)

In [ ]:
model.remove_output_netcdfs()

In [ ]:
# remove images
for filename in filenames:
    os.remove(filename)